<h1> Data Preprocessing </h1>
<hr>
<h3> In this section we will first get a quick glimpse of the dataset that we are going to analyze and see how we can make it better and more efficient for further analysis </h3>
<br><br>
Let's start by importing the necessary libraries and the dataset we are going to work on. Since we have the 'InvoiceDate' attribute in an undesirable format, we fix it by formatting this date variable in our desired form.
We also have some incorrect spellings and extra whitespaces in our strings that we would like to avoid and for the next steps we would like our strings to be in uppercase:

In [60]:
import numpy as np, pandas as pd, re, scipy as sp, scipy.stats

#Importing Dataset
pd.options.mode.chained_assignment = None
dataset = r'../Original-Dataset/Online Retail.csv'
Data = pd.read_csv(dataset, dtype= {'CustomerID': 'Int64'})
Data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [61]:
#Formatting Date/Time
Data['InvoiceDate'] = pd.to_datetime(Data['InvoiceDate'], format = '%m/%d/%Y %H:%M', box = False)

#Strings
Data['Description'] = Data['Description'].str.replace('.','').str.upper().str.strip()
Data['Description'] = Data['Description'].replace('\s+',' ',regex = True)
Data['InvoiceNo'] = Data['InvoiceNo'].astype(str).str.upper()
Data['StockCode'] = Data['StockCode'].str.upper()
Data['Country'] = Data['Country'].str.upper()
Data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,UNITED KINGDOM
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,UNITED KINGDOM
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART,6,2010-12-01 08:26:00,3.39,17850,UNITED KINGDOM


In [67]:
#Listing Some Irrelevant StockCodes
Irrelevant = Data['StockCode'].unique()
Irrelevant.sort()
print('Irrelevant Transactions: \n',Irrelevant[::-1][:4])
#Quantity and UnitPrice Summary
Data.describe().iloc[:,:2]

Irrelevant Transactions: 
 ['S' 'POST' 'PADS' 'M']


,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


As it is obvious in the summary of our dataset, we have some odd and irregular values in the 'UnitPrice' and 'Quantity' columns that we will find and remove to prevent them from negatively affect our analysis. In the 'StockCode' variable we can see that some of the transaction are not actually products, but they are some costs or fees regarding to the post or bank or other tansactions that we don't really need in our data.

In [68]:
#Outliers and Irrelevant Values
#Dropping all stockcodes that contain only strings
CodeTypes = list(map(lambda codes: any(char.isdigit() for char in codes), Data['StockCode']))
IrrelevantCodes = [i for i,v in enumerate(CodeTypes) if v == False]
Data.drop(IrrelevantCodes , inplace = True)
#Removing Outliers Based on Z-score
Data = Data[(np.abs(sp.stats.zscore(Data['UnitPrice']))<3) & (np.abs(sp.stats.zscore(Data['Quantity']))<5)]

We know that some of these transactions contains returned products and in those transaction the 'InvoiceNo' contains a 'c' character in the beginning and the 'UnitPrice' should have a negative value. But in our data we have purchases that have negative 'UnitPrice' and vice versa and we need to remove these errors. There are also some touples in which the 'UnitPrice' is not specified.

In the 'Description' attribute there are a lot of missing or incorrect values. In order to fix this problem, we will remove the transactions that have no available description at all and for the other missing values we check the 'Description' based on the product 'StockCode' and fill the missing values with the correct 'Description' that is available from the other transactions of the same 'StockCode'.

In [69]:
# Missing & Incorrect Values
Data.drop(Data[(Data.Quantity>0) & (Data.InvoiceNo.str.contains('C') == True)].index, inplace = True)
Data.drop(Data[(Data.Quantity<0) & (Data.InvoiceNo.str.contains('C') == False)].index, inplace = True)
Data.drop(Data[Data.Description.str.contains('?',regex=False) == True].index, inplace = True)
Data.drop(Data[Data.UnitPrice == 0].index, inplace = True)

for index,value in Data.StockCode[Data.Description.isna()==True].items():
    if pd.notna(Data.Description[Data.StockCode == value]).sum() != 0:
        Data.Description[index] = Data.Description[Data.StockCode == value].mode()[0]
    else:
        Data.drop(index = index, inplace = True)
        
Data['Description'] = Data['Description'].astype(str)

In this dataset, there are some rows that the same products have different 'UnitPrice'. This could come from the fact that there might be some discounts for some products or some special conditions for specific customers. We can also see that for the same customer we have two different price on two different days or different prices for different customer on the same day. So for the sake of this demonstration, we assume that these values are incorrect due to input errors or human mistakes and for each product we use the mode value of its 'UnitPrice'.

In [71]:
#Incorrect Prices
StockList = Data.StockCode.unique()
CalculatedMode = map(lambda x: Data.UnitPrice[Data.StockCode == x].mode()[0],StockList)
StockModes = list(CalculatedMode)
for i,v in enumerate(StockList):
    Data.loc[Data['StockCode']== v, 'UnitPrice'] = StockModes[i]

At last, we would like to add useful features for future analysis. The features that we will add for now are Final price of each transaction and the month and the day of the week in which the transaction took place, that we can take from the 'InvoiceDate' attribute.


There are also some incorrect customer IDs that for two different countries we have the same customer ID. We will fix the duplicate values by grouping the dataframe by 'CustomerID' and if any customer belongs to more than two countries, we replace the incorrect value with the mode value of the customer's country.

In [90]:
#Customers with Different Countries
Customers = Data.groupby('CustomerID')['Country'].unique()
Customers.loc[Customers.apply(lambda x:len(x)>1)]

CustomerID
12370           [CYPRUS, AUSTRIA]
12394          [BELGIUM, DENMARK]
12417            [BELGIUM, SPAIN]
12422    [AUSTRALIA, SWITZERLAND]
12429          [DENMARK, AUSTRIA]
12431        [AUSTRALIA, BELGIUM]
12455             [CYPRUS, SPAIN]
12457       [SWITZERLAND, CYPRUS]
Name: Country, dtype: object

In [91]:
#Fixing Duplicate CustomerIDs
for i,v in Data.groupby('CustomerID')['Country'].unique().items():
    if len(v)>1:
        Data.Country[Data['CustomerID'] == i] = Data.Country[Data['CustomerID'] == i].mode()[0]

#Adding Desired Features
Data['FinalPrice'] = Data['Quantity']*Data['UnitPrice']
Data['InvoiceMonth'] = Data['InvoiceDate'].apply(lambda x: x.strftime('%B'))
Data['Day of week'] = Data['InvoiceDate'].dt.day_name()

#Exporting Processed Data
Data.to_csv(r'../Cleaned-Dataset/OnlineRetail_Cleaned.csv', date_format = '%Y-%m-%d %H:%M', index = False)